In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Import from Part 2
from v2_part2_alice import KeyXORLayer, ResidualConvBlock

# ============ Simplified Bob Decoder ============
class SimplifiedBobDecoder(nn.Module):
    """
    Bob: Decrypts encrypted messages using correct keys
    Mirror architecture of Alice
    """
    def __init__(self, vocab_size=98, embed_dim=128, num_layers=3, max_len=128):
        super(SimplifiedBobDecoder, self).__init__()

        self.embed_dim = embed_dim
        self.vocab_size = vocab_size

        # Input projection
        self.input_proj = nn.Linear(embed_dim, embed_dim)

        # Decoding layers with key-XOR (reverse of Alice)
        self.key_xor_layers = nn.ModuleList([
            KeyXORLayer(embed_dim) for _ in range(num_layers)
        ])

        self.conv_layers = nn.ModuleList([
            ResidualConvBlock(embed_dim) for _ in range(num_layers)
        ])

        # Output head to predict characters
        self.output_head = nn.Sequential(
            nn.Linear(embed_dim, embed_dim),
            nn.LayerNorm(embed_dim),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(embed_dim, vocab_size)
        )

        # Initialize weights
        self._init_weights()

    def _init_weights(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                module.weight.data.normal_(mean=0.0, std=0.02)
                if module.bias is not None:
                    module.bias.data.zero_()

    def forward(self, encrypted, keys):
        """
        encrypted: (batch, seq_len, embed_dim) - from Alice
        keys: list of 4 tensors, each (batch, seq_len)
        Returns: (batch, seq_len, vocab_size) - character logits
        """
        x = self.input_proj(encrypted)

        # Reverse the encryption layers (reverse order and operations)
        for i in reversed(range(len(self.conv_layers))):
            # Apply key-XOR (must have correct keys!)
            x = self.key_xor_layers[i](x, keys[i])

            # Deconvolutional decoding
            x_conv = x.permute(0, 2, 1)  # (batch, embed_dim, seq_len)
            x_conv = self.conv_layers[i](x_conv)
            x = x_conv.permute(0, 2, 1)  # (batch, seq_len, embed_dim)

        # Predict characters
        logits = self.output_head(x)

        return logits


# ============ Simplified Eve Attacker ============
class SimplifiedEveAttacker(nn.Module):
    """
    Eve: Tries to decrypt WITHOUT keys
    Uses different architecture (no key-XOR layers)
    """
    def __init__(self, vocab_size=98, embed_dim=128, num_layers=4, max_len=128):
        super(SimplifiedEveAttacker, self).__init__()

        self.embed_dim = embed_dim

        # Eve uses more layers but no key information
        self.input_proj = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * 2),
            nn.LayerNorm(embed_dim * 2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(embed_dim * 2, embed_dim),
            nn.LayerNorm(embed_dim),
            nn.ReLU()
        )

        # Powerful convolutional layers
        self.conv_layers = nn.ModuleList([
            ResidualConvBlock(embed_dim) for _ in range(num_layers)
        ])

        # Output head
        self.output_head = nn.Sequential(
            nn.Linear(embed_dim, embed_dim),
            nn.LayerNorm(embed_dim),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(embed_dim, vocab_size)
        )

        # Initialize weights
        self._init_weights()

    def _init_weights(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                module.weight.data.normal_(mean=0.0, std=0.02)
                if module.bias is not None:
                    module.bias.data.zero_()

    def forward(self, encrypted):
        """
        encrypted: (batch, seq_len, embed_dim)
        NO keys - Eve must break it without keys
        Returns: (batch, seq_len, vocab_size)
        """
        x = self.input_proj(encrypted)

        # Process through conv layers (no key access!)
        for conv_layer in self.conv_layers:
            x_conv = x.permute(0, 2, 1)
            x_conv = conv_layer(x_conv)
            x = x_conv.permute(0, 2, 1)

        # Predict characters
        logits = self.output_head(x)

        return logits


# ============ Bob Wrapper ============
class BobDecryptor:
    """Wrapper for Bob decoder"""
    def __init__(self, vocab_size=98, embed_dim=128, num_layers=3, max_len=128,
                 device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.bob = SimplifiedBobDecoder(vocab_size, embed_dim, num_layers, max_len).to(device)

        from v2_part1_simplified import StringProcessor
        self.processor = StringProcessor(max_length=max_len)

    def decrypt(self, encrypted, keys):
        """
        Decrypt encrypted embeddings
        encrypted: (batch, seq_len, embed_dim)
        keys: dict from KeyManager
        """
        # Truncate keys to match sequence length
        seq_len = encrypted.size(1)
        truncated_keys = [k[:, :seq_len] for k in keys['key_tensors']]

        # Decrypt
        self.bob.eval()
        with torch.no_grad():
            logits = self.bob(encrypted, truncated_keys)
            tokens = torch.argmax(logits, dim=-1)

        # Convert to strings
        messages = self.processor.batch_decode(tokens)

        return messages, tokens, logits


# ============ Eve Wrapper ============
class EveAttacker:
    """Wrapper for Eve attacker"""
    def __init__(self, vocab_size=98, embed_dim=128, num_layers=4, max_len=128,
                 device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.eve = SimplifiedEveAttacker(vocab_size, embed_dim, num_layers, max_len).to(device)

        from v2_part1_simplified import StringProcessor
        self.processor = StringProcessor(max_length=max_len)

    def attack(self, encrypted):
        """
        Try to decrypt without keys
        encrypted: (batch, seq_len, embed_dim)
        """
        # Attack without keys
        self.eve.eval()
        with torch.no_grad():
            logits = self.eve(encrypted)
            tokens = torch.argmax(logits, dim=-1)

        # Convert to strings
        messages = self.processor.batch_decode(tokens)

        return messages, tokens, logits


# ============ Usage Example ============
if __name__ == "__main__":
    print("="*70)
    print("SIMPLIFIED BOB DECODER & EVE ATTACKER V2")
    print("="*70)

    # Initialize
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    from v2_part1_simplified import StringProcessor, KeyManager

    processor = StringProcessor(max_length=64)
    bob_dec = BobDecryptor(
        vocab_size=processor.vocab_size,
        embed_dim=128,
        num_layers=3,
        max_len=64,
        device=device
    )
    eve_att = EveAttacker(
        vocab_size=processor.vocab_size,
        embed_dim=128,
        num_layers=4,
        max_len=64,
        device=device
    )

    # Simulate encrypted data
    batch_size = 3
    seq_len = 64
    embed_dim = 128
    encrypted = torch.randn(batch_size, seq_len, embed_dim).to(device)

    # Generate keys
    key_manager = KeyManager(device=device)
    keys = key_manager.generate_keys_for_batch(batch_size, seq_len)

    print("\nBob decrypting with correct keys...")
    bob_messages, _, _ = bob_dec.decrypt(encrypted, keys)
    print("Bob's outputs:")
    for i, msg in enumerate(bob_messages):
        print(f"  {i+1}. '{msg}'")

    print("\nEve attacking without keys...")
    eve_messages, _, _ = eve_att.attack(encrypted)
    print("Eve's outputs:")
    for i, msg in enumerate(eve_messages):
        print(f"  {i+1}. '{msg}'")

    print("\n" + "="*70)

SIMPLIFIED BOB DECODER & EVE ATTACKER V2

Bob decrypting with correct keys...
Bob's outputs:
  1. 'R@U)K@y@@@@@@@)0@@@@K@RU@UC@@@w;K)C@@@j00;@oR0@@jc@C00@U@UC@R@9Z'
  2. 'R0R@@wUR@@@0@@R@@0R @W;0@)URR@)0@0URR@URZ00@w0w0UUR09FwU@jcx)@;w'
  3. 'w@@RR000)@@R@R00A0U@@@);@UU)0URcRR@U;R@C@9 0@URU0U@x@@@@)0)RUU)x'

Eve attacking without keys...
Eve's outputs:
  1. '7SSnSSSSS!SSnSSSSSSSSSSSSnSSSSSSSSSSnSSSSSS0SSSSSSSSSSSnSSSSSSSS'
  2. 'gSSSSS7SSS7SSSnSSSSnSSSS7SS7SSnSSnnSSSSSSnSS7SSSn7{SnSSSSSS7SSS0'
  3. 'S7SSSnSSSSSSSSSSSSSSSSSSSS77S7SSnSnSSSSSSSSSSnnSSSSSn7SSSSSS!SS:'

